In [1]:
import pandas as pd
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from collections import Counter
import torch
import gensim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from nexus_ai.sentence_sentiment_analysis import multi_class_model
from nexus_ai.sentence_sentiment_analysis import model
from nexus_ai.sentence_sentiment_analysis.sentence_sentiment_analysis_model import train, test, predict
import numpy as np
from nexus_ai.sentence_sentiment_analysis.preprocessing import clean_reviews, creat_vocab, tokenize_data, pad_features, train_test_split, google_clean_reviews, clean_reviews_and_creat_vocab

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meshari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('dataset/ar_reviews_100k.tsv',sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [3]:
print(f'number of null values \n{df.isnull().sum()}')
print(f'number of duplicate values is {df.duplicated().sum()}')

number of null valueslabel    0
text     0
dtype: int64
number of duplicate values0


In [4]:
labels = df['label'].unique()
print(f'labels {labels}')

labels ['Positive' 'Mixed' 'Negative']


let's tranform Mixed into Neutral

In [5]:
print(f'dataset shape before transforming: {df.shape}')
df.loc[df[df['label']=='Mixed'].index, 'label'] = 'Neutral'
# print(len((df[df['label']=='Mixed']).index.to_list()))
# print(df.loc[df[(df['label']=='Mixed')].index, :])
print(f'dataset shape after transforming: {df.shape}')
labels = df['label'].unique()
print(f'labels {labels}')

dataset shape before transforming: (99999, 2)
dataset shape after transforming: (99999, 2)
labels ['Positive' 'Neutral' 'Negative']


In [6]:
pos_prec = df[df['label'] == 'Positive'].shape[0]
neg_prec = df[df['label'] == 'Negative'].shape[0]
nat_prec = df[df['label'] == 'Neutral'].shape[0]
print(f'Positive precentage {pos_prec}')
print(f'Negative precentage {neg_prec}')
print(f'Neutral precentage {nat_prec}')

Positive precentage 33333
Negative precentage 33333
Neutral precentage 33333


the dataset is balanced which is very good

now let's use the camel tools to preproccess the data

In [7]:
print('before unicode normalizing')
print(df.loc[:3, 'text'].to_list())
df['text'] = df['text'].apply(lambda x: normalize_unicode(x))

print('after unicode normalizing')
print(df.loc[:3, 'text'].to_list())

before unicode normalizing
['ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم', 'أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة', 'هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطة مشاعر انسانية حقيقية لدرجة غريبة.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيق

In [8]:
print('oringinal text')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: normalize_alef_ar(x))
print('text after Normalizing alef variants to (ا)')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: normalize_alef_maksura_ar(x))
print('text after Normalizing alef maksura (ى) to yeh (ي)')
print(df.loc[:3, 'text'].to_list())

df['text'] = df['text'].apply(lambda x: normalize_teh_marbuta_ar(x))
print('text after Normalizing teh marbuta (ة) to heh (ه)')
print(df.loc[:3, 'text'].to_list())

oringinal text
['ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم', 'أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة', 'هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطة مشاعر انسانية حقيقية لدرجة غريبة.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيقي تثق فيما س

In [9]:
print('oringinal text')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: dediac_ar(x))
print('text after removing Arabic diacritical marks')
print(df.loc[:3, 'text'].to_list())

oringinal text
['ممتاز نوعا ما . النظافه والموقع والتجهيز والشاطيء. المطعم', 'احد اسباب نجاح الامارات ان كل شخص في هذه الدوله يعشق ترابها. نحن نحب الامارات. ومضات من فكر. نصائح لدوله تطمح بالصفوف الاولي و قائد لا يقبل الا براحه شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءه مرات كثيره', 'هادفه .. وقويه. تنقلك من صخب شوارع القاهره الي هدوء جبال الشيشان .. للتعرف علي حقيقه ما يجري في تلك البلاد من حروب ضاربه بحق المسلمين و جزء كبير من تاريخ تلك المنطقه. التضحيه .. الرجوله .. الوفاء والكثير من القيم الاخري اثبتت وجودها في تلك الروايه البسيطه', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطي مرحله ان القارئ يخلص الروايه وهو فاتح بؤه لمرحله ان القارئ يخلص الروايه وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبه ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطه مشاعر انسانيه حقيقيه لدرجه غريبه.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيقي تثق فيما س

sadly even in the sample original text there was no Arabic diacritical marks, but we now that it did what it should do

In [10]:
print('oringinal text')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: simple_word_tokenize(x))
print('text after splitting')
print(df.loc[:3, 'text'].to_list())

oringinal text
['ممتاز نوعا ما . النظافه والموقع والتجهيز والشاطيء. المطعم', 'احد اسباب نجاح الامارات ان كل شخص في هذه الدوله يعشق ترابها. نحن نحب الامارات. ومضات من فكر. نصائح لدوله تطمح بالصفوف الاولي و قائد لا يقبل الا براحه شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءه مرات كثيره', 'هادفه .. وقويه. تنقلك من صخب شوارع القاهره الي هدوء جبال الشيشان .. للتعرف علي حقيقه ما يجري في تلك البلاد من حروب ضاربه بحق المسلمين و جزء كبير من تاريخ تلك المنطقه. التضحيه .. الرجوله .. الوفاء والكثير من القيم الاخري اثبتت وجودها في تلك الروايه البسيطه', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطي مرحله ان القارئ يخلص الروايه وهو فاتح بؤه لمرحله ان القارئ يخلص الروايه وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبه ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطه مشاعر انسانيه حقيقيه لدرجه غريبه.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيقي تثق فيما س

In [11]:
# get tokens for pretrained model
embd_model = gensim.models.Word2Vec.load('word2vec/word2vec.model')
reviews = df['text'].tolist()
reviews_ints = []
for review in reviews:
    # 662109 is the new unknown token added: check the word2vec_add_unk_token notebook to see the procees
    reviews_ints.append([embd_model.wv.key_to_index.get(word, 662109) for word in review])

In [12]:
print(reviews[0])
print(reviews_ints[0])

['ممتاز', 'نوعا', 'ما', '.', 'النظافه', 'والموقع', 'والتجهيز', 'والشاطيء', '.', 'المطعم']
[10346, 1660, 27, 0, 20203, 26486, 62562, 659320, 0, 12469]


In [13]:
labels = df['label'].tolist()
encoded_labels = []
for label in labels:
    if label == 'Positive':
        encoded_labels.append(2)
    if label == 'Neutral':
        encoded_labels.append(1)
    elif label == 'Negative':
        encoded_labels.append(0) 

unique, counts = np.unique(encoded_labels, return_counts=True)
distribution = dict(zip(unique, counts))
print('encoded_labels class distribution:')
for item in distribution.items():
    print(item)

encoded_labels class distribution:
(0, 33333)
(1, 33333)
(2, 33333)


In [14]:
features = pad_features(reviews_ints, 256)

In [15]:
print(features.shape)

(99999, 256)


In [16]:
train_x, validate_x, test_x, train_y, validate_y, test_y = train_test_split(features, encoded_labels, numclasses=3, train_frac=0.8, balanced=True)

make sure the classes (labels) are sorted to have a balanced splits


In [17]:
unique, counts = np.unique(train_y, return_counts=True)
distribution = dict(zip(unique, counts))
print('train_y class distribution:')
for item in distribution.items():
    print(item)

unique, counts = np.unique(validate_y, return_counts=True)
distribution = dict(zip(unique, counts))
print('validate_y class distribution:')
for item in distribution.items():
    print(item)

unique, counts = np.unique(test_y, return_counts=True)
distribution = dict(zip(unique, counts))
print('test_y class distribution:')
for item in distribution.items():
    print(item)

train_y class distribution:
(0, 26666)
(1, 26667)
(2, 26667)
validate_y class distribution:
(0, 3333)
(1, 3333)
(2, 3333)
test_y class distribution:
(0, 3333)
(1, 3333)
(2, 3333)


In [18]:
print(train_x.shape, train_y.shape)
print(validate_x.shape, validate_y.shape)
print(test_x.shape, test_y.shape)

(80000, 256) (80000,)
(9999, 256) (9999,)
(9999, 256) (9999,)


In [19]:
#creating data loaders to be used in training and testing 
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(validate_x), torch.from_numpy(validate_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#batch_size for dataloaders
batch_size = 256

num_worker = 0

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True, num_workers=num_worker)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True, num_workers=num_worker)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True, num_workers=num_worker)

since the dataset are not very large let's try a few architecture, while doing a simple grid search for each architecture optimal learning rate

In [20]:
lr_list = [
    1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3,
    1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 7e-4, 8e-4, 9e-4
]
# lr_list = [
#     1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5, 1e-6,
# ]
for lr in lr_list:
    torch.cuda.empty_cache()    
    #make sure the gpu is avalibale before moving the data to It else the work would be done using the cpu
    train_on_gpu=torch.cuda.is_available()

    #specifing the model hypermetrs and creating the model
    vocab_size = len(embd_model.wv.key_to_index)
    output_size = 3
    embedding_dim = 300
    hidden_dim = 100
    n_layers = 2
    drop_prob = 0.3
    
    net = multi_class_model.ar_word2vec_RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, train_on_gpu=train_on_gpu,
                    bidirectional=False, drop_prob=drop_prob, train_embd=True)

    print(net)

    # optimization functions
    # optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.00001)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    print(f'training with learning rate of {lr}....')
    # training the model
    train(net, train_loader, valid_loader, optimizer, embd_model.wv.key_to_index, batch_size, multi_class=True, seq_length=256, epochs=10,
        print_every=1, train_on_gpu=train_on_gpu, save_dic='models/arabic_01.pth')
    print('\n\n\n\n')
    torch.cuda.empty_cache()

ar_word2vec_RNN(
  (embd): Embedding(662110, 300)
  (lstm): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=100, out_features=3, bias=True)
  (LogSoftmax): LogSoftmax(dim=1)
)
training with learning rate of 0.001....
Epoch: 1/10... Step: 312... Loss: 1.099592... Val Loss: 1.098465
Validation loss decreased (inf --> 1.098465).  Saving model ...
Epoch: 2/10... Step: 624... Loss: 1.064780... Val Loss: 1.047024
Validation loss decreased (1.098465 --> 1.047024).  Saving model ...
Epoch: 3/10... Step: 936... Loss: 0.889849... Val Loss: 0.856504
Validation loss decreased (1.047024 --> 0.856504).  Saving model ...
Epoch: 4/10... Step: 1248... Loss: 0.850127... Val Loss: 0.833803
Validation loss decreased (0.856504 --> 0.833803).  Saving model ...
Epoch: 5/10... Step: 1560... Loss: 0.720864... Val Loss: 0.773401
Validation loss decreased (0.833803 --> 0.773401).  Saving model ...
Epoch: 6/10... Step: 1872... Loss

In [25]:
torch.cuda.empty_cache()    
# make sure the gpu is avalibale before moving the data to It else the work would be done using the cpu
train_on_gpu=torch.cuda.is_available()

# specifing the model hypermetrs and creating the model
vocab_size = len(embd_model.wv.key_to_index)
output_size = 3
embedding_dim = 300
hidden_dim = 100
n_layers = 2
drop_prob = 0.3

net = multi_class_model.ar_word2vec_RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, train_on_gpu=train_on_gpu,
            bidirectional=False, drop_prob=drop_prob)

print(net)

lr = 7e-4
# optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.00001)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

ar_word2vec_RNN(
  (embd): Embedding(662110, 300)
  (lstm): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=100, out_features=3, bias=True)
  (LogSoftmax): LogSoftmax(dim=1)
)


In [26]:
# training the model
train(net, train_loader, valid_loader, optimizer, embd_model.wv.key_to_index, batch_size, multi_class=True, seq_length=256, epochs=10,
print_every=1, train_on_gpu=train_on_gpu, save_dic='models/arabic_01.pth')
torch.cuda.empty_cache()

Epoch: 1/10... Step: 312... Loss: 1.098104... Val Loss: 1.098547
Validation loss decreased (inf --> 1.098547).  Saving model ...
Epoch: 2/10... Step: 624... Loss: 1.016623... Val Loss: 1.028347
Validation loss decreased (1.098547 --> 1.028347).  Saving model ...
Epoch: 3/10... Step: 936... Loss: 0.887928... Val Loss: 0.924211
Validation loss decreased (1.028347 --> 0.924211).  Saving model ...
Epoch: 4/10... Step: 1248... Loss: 0.814031... Val Loss: 0.812846
Validation loss decreased (0.924211 --> 0.812846).  Saving model ...
Epoch: 5/10... Step: 1560... Loss: 0.737314... Val Loss: 0.778060
Validation loss decreased (0.812846 --> 0.778060).  Saving model ...
Epoch: 6/10... Step: 1872... Loss: 0.714753... Val Loss: 0.765691
Validation loss decreased (0.778060 --> 0.765691).  Saving model ...
Epoch: 7/10... Step: 2184... Loss: 0.732823... Val Loss: 0.748772
Validation loss decreased (0.765691 --> 0.748772).  Saving model ...
Epoch: 8/10... Step: 2496... Loss: 0.706392... Val Loss: 0.7368

In [27]:
# testing the model
test(test_loader, train_on_gpu, batch_size, multi_class=True, net=net)

Test loss: 0.744
Test accuracy: 0.658


In [28]:
# testing the last saved model
test(test_loader, train_on_gpu, batch_size, multi_class=True, model_path='models/arabic_01.pth')

Test loss: 0.733
Test accuracy: 0.658
